In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
# for filename in filenames:
#       print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
image_path = '/kaggle/input/kermany2018/oct2017/OCT2017 '
oct_csv_path = '/kaggle/input/oct-csv/'
train_dir = image_path + "/train/"
valid_dir = image_path + "/val/"
test_dir = image_path + "/test/"

In [3]:
classes = ['CNV', 'DME', 'DRUSEN', 'NORMAL']
cols = [x.upper() for x in classes]
dirs = [train_dir, valid_dir, test_dir]
label = {0: 'CNV', 1: 'DME', 2: 'DRUSEN', 3: 'NORMAL'}
IMG_SIZE = 224

# if we should read the directory structre, if False then use the CSV files already saved
# Once you generate the csv files you should probably download them and re-upload into kaggle and set this to FALSE
REGEN = False

In [4]:
def create_df (path, classes=classes):
  df = pd.DataFrame(columns=['FILENAME', 'CNV', 'DME', 'DRUSEN', 'NORMAL'])
  for sub_dir in classes:
    condition = {'NORMAL': 0, 'CNV': 0, 'DME':0, 'DRUSEN': 0}
    files = os.listdir(path + sub_dir)
    if (sub_dir== 'NORMAL'):
      condition['NORMAL'] = 1
    elif (sub_dir == 'CNV'):
      condition['CNV'] = 1
    elif (sub_dir == 'DME'):
      condition['DME'] = 1
    else:
      condition['DRUSEN']= 1
    for f in files:
      df = df.append({'FILENAME': path +  sub_dir  + "/" + f, 
                      'NORMAL': condition['NORMAL'], 
                      'CNV': condition['CNV'],
                      'DME': condition['DME'],
                      'DRUSEN': condition['DRUSEN']}, ignore_index=True)
  return df

In [5]:
# Generting the DataFrames of the filenames
# this is primarily used so we can sub-sample files easier for the different training strategies
if (REGEN):
  train_df = create_df(train_dir)
  valid_df = create_df(valid_dir)
  test_df = create_df(test_dir)
  train_df.to_csv("train_data.csv")
  valid_df.to_csv("valid_data.csv")
  test_df.to_csv("test_data.csv")
else:
  train_df = pd.read_csv(oct_csv_path + "train_data.csv")
  valid_df = pd.read_csv(oct_csv_path + "valid_data.csv")
  test_df = pd.read_csv(oct_csv_path + "test_data.csv")

In [6]:
print ("Training Data: ", train_df.shape)
print ("Validation Data: ", valid_df.shape)
print ("Test Data: ", test_df.shape)

Training Data:  (83484, 5)
Validation Data:  (36, 5)
Test Data:  (972, 5)


In [7]:
train_df.head()

,FILENAME,CNV,DME,DRUSEN,NORMAL
0,/kaggle/input/kermany2018/oct2017/OCT2017 /tra...,1,0,0,0
1,/kaggle/input/kermany2018/oct2017/OCT2017 /tra...,1,0,0,0
2,/kaggle/input/kermany2018/oct2017/OCT2017 /tra...,1,0,0,0
3,/kaggle/input/kermany2018/oct2017/OCT2017 /tra...,1,0,0,0
4,/kaggle/input/kermany2018/oct2017/OCT2017 /tra...,1,0,0,0


In [8]:
# Printing out the # of samples for each subsample percentage 
print ("Trainig Data percentages:")
print (" 1% ==> ", int(.01 * train_df.shape[0]))
print (" 5% ==> ", int(.05 * train_df.shape[0]))
print ("10% ==> ", int(.1  * train_df.shape[0]))
print ("25% ==> ", int(.25 * train_df.shape[0]))
print ("50% ==> ", int(.5  * train_df.shape[0]))
print ("75% ==> ", int(.75 * train_df.shape[0]))
print ("90% ==> ", int(.9  * train_df.shape[0]))
print ("98% ==> ", int(.98 * train_df.shape[0]))

Trainig Data percentages:
 1% ==>  834
 5% ==>  4174
10% ==>  8348
25% ==>  20871
50% ==>  41742
75% ==>  62613
90% ==>  75135
98% ==>  81814


In [9]:
# Sampling 100% of the data
sample = train_df.sample(frac=1.0, random_state=10, axis=0)
sample.shape

(83484, 5)

In [10]:
# determine class weights to feed into neural network during training
def get_classweight(df):
  total = df.shape[0]
  num_norm = df['NORMAL'].sum()
  num_cnv = df['CNV'].sum()
  num_dme = df['DME'].sum()
  num_drusen = df['DRUSEN'].sum()
  norm_weight = (1/num_norm) * (total/4)
  cnv_weight = (1/num_cnv) * (total/4)
  dme_weight = (1/num_dme) * (total/4)
  drusen_weight = (1/num_drusen) * (total/4)
  class_weight = {0 : cnv_weight, 1: dme_weight,
                  2 : drusen_weight, 3: norm_weight}
  return class_weight

In [11]:
class_weight = get_classweight(sample)
class_weight

{0: 0.5609729875016799,
 1: 1.839178709904829,
 2: 2.4223537604456826,
 3: 0.7931217936538096}

In [12]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_addons as tfa
import tensorflow.keras.applications as app
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [13]:
train_image_datagen = ImageDataGenerator(rotation_range=90, width_shift_range=[-.1,.1], height_shift_range=[-.1,.1],
                                         shear_range=0.25, zoom_range=0.3, horizontal_flip=True,
                                         vertical_flip=True, rescale = 1./255., validation_split=0.1)

# Setting the imgages to come from the dataframe where we specify the filenames and columns to use for "labels"
train_imgs = train_image_datagen.flow_from_dataframe(sample, directory=None, x_col='FILENAME', y_col=cols, subset="training",
                                        class_mode="raw", target_size=(IMG_SIZE,IMG_SIZE), batch_size=32, seed=10)
valid_imgs = train_image_datagen.flow_from_dataframe(sample, directory=None, x_col='FILENAME', y_col=cols, subset="validation",
                                        class_mode="raw", target_size=(IMG_SIZE,IMG_SIZE), batch_size=32, seed=10)

Found 75136 validated image filenames.
Found 8348 validated image filenames.


In [14]:
# Creating the model based on Xception Network
input_layer = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
base_model = app.xception.Xception(include_top=False, weights="imagenet", input_shape=(IMG_SIZE,IMG_SIZE,3))
base_model.trainable = False

x = base_model(input_layer, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
output = keras.layers.Dense(4, activation="softmax")(x)

model = keras.Model(inputs=input_layer, outputs=output)
model.summary()

83689472/83683744 [==============================] - 0s 0us/step
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
xception (Functional)        (None, 7, 7, 2048)        20861480  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 4)                 8196      
Total params: 20,869,676
Trainable params: 8,196
Non-trainable params: 20,861,480
_________________________________________________________________


In [15]:
# This code did not work, it caused I/O Error 5:
# model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3), loss='categorical_crossentropy', metrics='accuracy')
model.compile(optimizer=keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=[tfa.metrics.F1Score(4,"micro"), 'accuracy'])

In [16]:
# Creating a checkpoint to save the best model so that we can reload it once training is complete
checkpoint_cb = keras.callbacks.ModelCheckpoint("oct_xception100per.h5", save_best_only=True)
# Adding an an early stop callback to avoid overfitting in case the model is not improving after 5 consescutive epochs
earlystop_cb = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

In [17]:
history = model.fit(train_imgs,  steps_per_epoch=1000, epochs=100, verbose=1, validation_data=valid_imgs, 
                    class_weight=class_weight, callbacks=[checkpoint_cb, earlystop_cb])

Epoch 1/100
1000/1000 [==============================] - 791s 782ms/step - loss: 1.0566 - f1_score: 0.5899 - accuracy: 0.5899 - val_loss: 0.8353 - val_f1_score: 0.6618 - val_accuracy: 0.6618
Epoch 2/100
1000/1000 [==============================] - 677s 677ms/step - loss: 0.8724 - f1_score: 0.6866 - accuracy: 0.6866 - val_loss: 0.7786 - val_f1_score: 0.6890 - val_accuracy: 0.6890
Epoch 3/100
1000/1000 [==============================] - 673s 673ms/step - loss: 0.8176 - f1_score: 0.7077 - accuracy: 0.7077 - val_loss: 0.7579 - val_f1_score: 0.7033 - val_accuracy: 0.7033
Epoch 4/100
1000/1000 [==============================] - 629s 628ms/step - loss: 0.7928 - f1_score: 0.7162 - accuracy: 0.7162 - val_loss: 0.7773 - val_f1_score: 0.6926 - val_accuracy: 0.6926
Epoch 5/100
1000/1000 [==============================] - 618s 617ms/step - loss: 0.7792 - f1_score: 0.7234 - accuracy: 0.7234 - val_loss: 0.7252 - val_f1_score: 0.7138 - val_accuracy: 0.7138
Epoch 6/100
1000/1000 [======================

In [18]:
test_image_datagen = ImageDataGenerator( rescale = 1./255.)

test_imgs = test_image_datagen.flow_from_dataframe(test_df, directory=None, x_col='FILENAME', y_col=cols, validate_filenames=True,
                                        class_mode="raw", target_size=(224,224), batch_size=32)

Found 968 validated image filenames.


/opt/conda/lib/python3.7/site-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 4 invalid image filename(s) in x_col="FILENAME". These filename(s) will be ignored.
  .format(n_invalid, x_col)


In [19]:
model.load_weights("oct_xception100per.h5")
model.evaluate(test_imgs)

31/31 [==============================] - 8s 266ms/step - loss: 0.3991 - f1_score: 0.8492 - accuracy: 0.8492


[0.39906707406044006, 0.8491734862327576, 0.8491735458374023]

In [20]:
results = model.predict(test_imgs)
results

array([[8.00621435e-02, 2.47314982e-02, 8.79140496e-01, 1.60658639e-02],
       [3.90859097e-01, 3.14673856e-02, 5.01452744e-01, 7.62207806e-02],
       [3.02537270e-02, 9.69362497e-01, 1.00427365e-04, 2.83292582e-04],
       ...,
       [3.92197907e-01, 1.09767430e-01, 4.87209767e-01, 1.08248908e-02],
       [8.71802092e-01, 1.20001048e-01, 8.09742324e-03, 9.93609719e-05],
       [7.64022814e-03, 9.86903787e-01, 5.33280056e-03, 1.23239835e-04]],
      dtype=float32)

In [21]:
from sklearn.metrics import classification_report, plot_confusion_matrix, confusion_matrix

In [22]:
model.save("oct_xception100per.h5")